In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pass
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
import pathlib
import cv2
from tensorflow import keras
from PIL import Image
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
CATEGORIES = ['Black-grass',
          'Charlock',
          'Cleavers',
          'Common Chickweed',
          'Common wheat',
          'Fat Hen',
          'Loose Silky-bent',
          'Maize',
          'Scentless Mayweed',
          'Shepherds Purse',
          'Small-flowered Cranesbill',
          'Sugar beet']

IMG_SIZE=300

DATADIR  = '../input/plant-seedlings-classification/train'
DATADIR_TEST  = '../input/plant-seedlings-classification/test'

In [ ]:
for category in CATEGORIES:
    path=os.path.join(DATADIR, category)
    for img in os.listdir(path):
        img_array=cv2.imread(os.path.join(path,img))
        plt.imshow(img_array)
        plt.show()
        break
    break

In [ ]:
training_data=[]

def create_training_data():
    for category in CATEGORIES:

        path=os.path.join(DATADIR, category)
        class_num=CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array=cv2.imread(os.path.join(path,img))
                new_array=cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
                
                # Pre-processing (via function pre_process) ##
                
#                 image = pre_process(new_array)
                image = new_array
                training_data.append([image,class_num])
            except Exception as e:
                print(e)
                pass
            
create_training_data()  

In [ ]:
import numpy as np

X_train=[]
y_train=[]

for categories, label in training_data:
    X_train.append(categories)
    y_train.append(label)
# X = np.array(X).reshape(train_len,-1)

y_train = np.array(y_train)

y_train.shape

In [ ]:
from keras.utils import np_utils

y_train = np_utils.to_categorical(y_train, num_classes=len(CATEGORIES))
# y_test = np_utils.to_categorical(y_test, num_classes=len(CATEGORIES))

X_train = np.array(X_train).astype('float32')

In [ ]:
print(X_train.shape)

print(y_train.shape)

In [ ]:
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

X_train, y_train = unison_shuffled_copies(X_train, y_train)

In [ ]:
y_train[51]

In [ ]:
def create_model(base_model):
    base_model.trainable = True
    global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
    prediction_layer = tf.keras.layers.Dense(12, activation='softmax')(global_average_layer)
    model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0001), loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), metrics=["accuracy"])
    return model

# define appropriate callbacks
def training_callbacks():
    
    # save best model regularly
    save_best_model = tf.keras.callbacks.ModelCheckpoint(filepath = 'model.h5',
        monitor = 'val_accuracy', save_best_only = True, verbose = 1)
    
    # reduce learning rate when it stops decreasing
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_accuracy', factor = 0.4,
                              patience = 3, min_lr = 1e-10, verbose = 1, cooldown = 1)
    
    # stop training early if no further improvement
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor = 'val_accuracy', min_delta = 1e-2, patience = 10, verbose = 1,
        mode = 'max', baseline = None, restore_best_weights = True
    )

    return [save_best_model, reduce_lr, early_stopping]

# function that returns the model with custom layers appended
def model1():
    width, height = IMG_SIZE,IMG_SIZE
    # define the input to the model
    input_model = tf.keras.layers.Input(shape = (width, height, 3), name = 'image_input')
    random_flip_model = tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal")(input_model)
    random_rotation_model = tf.keras.layers.experimental.preprocessing.RandomRotation(0.1)(random_flip_model)
    random_zoom_model = tf.keras.layers.experimental.preprocessing.RandomZoom(0.1)(random_rotation_model)
    main_model = tf.keras.applications.inception_resnet_v2.InceptionResNetV2(include_top = False, weights = 'imagenet')(random_zoom_model)
    main_model.trainable = True
    flattened_model = tf.keras.layers.Flatten()(main_model)
    dropout_1 = tf.keras.layers.Dropout(0.5)(flattened_model)
    dense_1 = tf.keras.layers.Dense(128, activation = 'relu', activity_regularizer=tf.keras.regularizers.l2(1e-5))(dropout_1)
    dropout_2 = tf.keras.layers.Dropout(0.5)(dense_1)
    output_model = tf.keras.layers.Dense(12, activation = "softmax", activity_regularizer=tf.keras.regularizers.l2(1e-5))(dropout_2)

    model = tf.keras.models.Model(input_model,  output_model)
    optimizer = tf.keras.optimizers.Adam(0.0005)
    
    # use categorical crossentropy loss since classification task
    model.compile(loss="categorical_crossentropy", optimizer = optimizer, metrics = ["accuracy"])
    return model

def model2():
    width, height = IMG_SIZE,IMG_SIZE
    # define the input to the model
    input_model = tf.keras.layers.Input(shape = (width, height, 3), name = 'image_input')
    random_flip_model = tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal")(input_model)
    random_rotation_model = tf.keras.layers.experimental.preprocessing.RandomRotation(0.1)(random_flip_model)
    random_zoom_model = tf.keras.layers.experimental.preprocessing.RandomZoom(0.1)(random_rotation_model)
    
    main_model = tf.keras.applications.EfficientNetB5(include_top = False, weights = 'imagenet')(random_zoom_model)
    main_model.trainable = True
    flattened_model = tf.keras.layers.Flatten()(main_model)
    dropout_1 = tf.keras.layers.Dropout(0.5)(flattened_model)
    dense_1 = tf.keras.layers.Dense(128, activation = 'relu', activity_regularizer=tf.keras.regularizers.l2(1e-5))(dropout_1)
    dropout_2 = tf.keras.layers.Dropout(0.5)(dense_1)
    output_model = tf.keras.layers.Dense(12, activation = "softmax", activity_regularizer=tf.keras.regularizers.l2(1e-5))(dropout_2)

    model = tf.keras.models.Model(input_model,  output_model)
    
    # use Adam optimizer with model
    optimizer = tf.keras.optimizers.Adam(0.0005)
    
    # use categorical crossentropy loss since classification task
    model.compile(loss="categorical_crossentropy", optimizer = optimizer, metrics = ["accuracy"])
    return model

def model3():
    width, height = IMG_SIZE,IMG_SIZE
    # define the input to the model
    input_model = tf.keras.layers.Input(shape = (width, height, 3), name = 'image_input')
    random_flip_model = tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal")(input_model)
    random_rotation_model = tf.keras.layers.experimental.preprocessing.RandomRotation(0.1)(random_flip_model)
    random_zoom_model = tf.keras.layers.experimental.preprocessing.RandomZoom(0.1)(random_rotation_model)
    main_model = tf.keras.applications.Xception(include_top = False, weights = 'imagenet')(random_zoom_model)
    main_model.trainable = True
    flattened_model = tf.keras.layers.Flatten()(main_model)
    dropout_1 = tf.keras.layers.Dropout(0.5)(flattened_model)
    dense_1 = tf.keras.layers.Dense(128, activation = 'relu', activity_regularizer=tf.keras.regularizers.l2(1e-5))(dropout_1)
    dropout_2 = tf.keras.layers.Dropout(0.5)(dense_1)
    output_model = tf.keras.layers.Dense(12, activation = "softmax", activity_regularizer=tf.keras.regularizers.l2(1e-5))(dropout_2)

    model = tf.keras.models.Model(input_model,  output_model)
    
    # use Adam optimizer with model
    optimizer = tf.keras.optimizers.Adam(0.0005)
    
    # use categorical crossentropy loss since classification task
    model.compile(loss="categorical_crossentropy", optimizer = optimizer, metrics = ["accuracy"])
    return model
                                                                                
def model4():
    width, height = IMG_SIZE,IMG_SIZE
    # define the input to the model
    input_model = tf.keras.layers.Input(shape = (width, height, 3), name = 'image_input')
    random_flip_model = tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal")(input_model)
    random_rotation_model = tf.keras.layers.experimental.preprocessing.RandomRotation(0.1)(random_flip_model)
    random_zoom_model = tf.keras.layers.experimental.preprocessing.RandomZoom(0.1)(random_rotation_model)
    main_model = tf.keras.applications.ResNet50(include_top = False, weights = 'imagenet')(random_zoom_model)
    main_model.trainable = True
    flattened_model = tf.keras.layers.Flatten()(main_model)
    dropout_1 = tf.keras.layers.Dropout(0.5)(flattened_model)
    dense_1 = tf.keras.layers.Dense(128, activation = 'relu', activity_regularizer=tf.keras.regularizers.l2(1e-5))(dropout_1)
    dropout_2 = tf.keras.layers.Dropout(0.5)(dense_1)
    output_model = tf.keras.layers.Dense(12, activation = "softmax", activity_regularizer=tf.keras.regularizers.l2(1e-5))(dropout_2)

    model = tf.keras.models.Model(input_model,  output_model)
    
    # use Adam optimizer with model
    optimizer = tf.keras.optimizers.Adam(0.0005)
    
    # use categorical crossentropy loss since classification task
    model.compile(loss="categorical_crossentropy", optimizer = optimizer, metrics = ["accuracy"])
    return model    

In [ ]:
# model1 = model1()
#model2 = model2()
# model3 = model3()
model4 = model4()

In [ ]:
batch_size = 16
epochs = 100

def fit_model(model):
    history = model.fit(X_train, y_train,
                        batch_size=batch_size,
                        steps_per_epoch=len(X_train)//batch_size, 
                        epochs=epochs, 
                        callbacks = training_callbacks(),
                        validation_split=0.1)
    return history

In [ ]:
# history1 = fit_model(model1)
# model1.save('models/model1.h5')
# history2 = fit_model(model2)
# model2.save('models/model2.h5')
# history3 = fit_model(model3)
# model3.save('models/model3.h5')
history4 = fit_model(model4)
model4.save('models/model4.h5')

In [ ]:
from IPython.display import FileLink
FileLink(r'models/model4.h5')

In [ ]:
testing_data =[]
def create_test_data():

    for img in os.listdir(DATADIR_TEST):
        try:
            img_array=cv2.imread(os.path.join(DATADIR_TEST,img))
            new_array=cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))

            ## Pre-processing (via function pre_process)

#             image = pre_process(new_array)
            image = new_array
            testing_data.append([image,img])
        except Exception as e:
            print(e)
            pass
create_test_data()

In [ ]:
import numpy as np

X_test=[]
y_test=[]

for categories, label in testing_data:
    X_test.append(categories)
    y_test.append(label)
# X = np.array(X).reshape(train_len,-1)

y_test = np.array(y_test)

y_test.shape

In [ ]:
X_test = np.array(X_test).astype('float32')

In [ ]:
models = []
model_names = ['efficientnet.h5', 'inceptionv2.h5', 'xception.h5']
for model_name in model_names:
    filename = os.path.join('../input/trained-models', model_name)
    model = tf.keras.models.load_model(filename)
    models.append(model)

In [ ]:
filename = os.path.join('../input/trained-models', 'xception.h5')
model = tf.keras.models.load_model(filename)

In [ ]:
predictions_classes_dict = {}
predictions_scores_dict = {}

In [ ]:
for i in range(len(models)):
    model_preds = models[i].predict(X_test)
    classes = []
    prediction_scores = []
    for data in range(0, model_preds.shape[0]):
        pred_index = model_preds[data, :].argmax(axis = -1)
        classes += [CATEGORIES[pred_index]]
        prediction_scores.append(max(model_preds[data]))
    predictions_classes_dict['model{}'.format(i+1)] = classes
    predictions_scores_dict['model{}'.format(i+1)] = prediction_scores

In [ ]:
predictions_classes_dict['species'] = []

In [ ]:
for i in range(len(predictions_dict['model1'])):
    class1 = predictions_classes_dict['model1'][i]
    class2 = predictions_classes_dict['model2'][i]
    class3 = predictions_classes_dict['model3'][i]
    ini_list = [class1, class2, class3]
    result = [item for items, c in Counter(ini_list).most_common() for item in [items] * c]
    
    if result[0] == result[1]:
        predictions_classes_dict['species'].append(result[0])
    else:
        # Get class w highest probability
        pred_scores = [predictions_scores_dict['model1'][i], predictions_scores_dict['model2'][i], predictions_scores_dict['model3'][i]]
        index = ini_list.index(max(ini_list))
        predictions_classes_dict['species'].append(ini_list[index])

In [ ]:
# generate submission csv
output_predictions = pd.DataFrame()
output_predictions['file'] = y_test
output_predictions['file'] = output_predictions['file'].str.replace(r'test/', '')
# output_predictions['model1'] = predictions_dict['model1']
# output_predictions['model2'] = predictions_dict['model2']
# output_predictions['model3'] = predictions_dict['model3']
output_predictions['species'] = predictions_dict['species']
output_predictions.to_csv('output2.csv', index = False)

In [ ]:
output_predictions[:50]

In [ ]:
# generate submission csv
output_predictions = pd.DataFrame()
output_predictions['file'] = y_test
output_predictions['file'] = output_predictions['file'].str.replace(r'test/', '')
output_predictions['species'] = classes
output_predictions.to_csv('output.csv', index = False)

In [ ]:
def load_all_models():
    all_models = []
    model_names = ['efficientnet.h5', 'inceptionv2.h5', 'xception.h5']
    for model_name in model_names:
        filename = os.path.join('../input/trained-models', model_name)
        model = tf.keras.models.load_model(filename)
        all_models.append(model)
        print('loaded:', filename)
    return all_models
models = load_all_models()
for i, model in enumerate(models):
    for layer in model.layers:
        layer.trainable = False

In [ ]:
count = 1
for model in models:
    for layer in model.layers:
        layer._name = layer.name + str(count)
        count = count + 1

In [ ]:
ensemble_visible = [model.input for model in models]
ensemble_outputs = [model.output for model in models]
merge = tf.keras.layers.concatenate(ensemble_outputs)
dense_1 = tf.keras.layers.Dense(128, activation = 'relu', activity_regularizer=tf.keras.regularizers.l2(1e-5))(merge)
dropout_2 = tf.keras.layers.Dropout(0.5)(dense_1)
output = tf.keras.layers.Dense(12, activation = "softmax", activity_regularizer=tf.keras.regularizers.l2(1e-5))(dropout_2)
model = tf.keras.models.Model(inputs=ensemble_visible, outputs=output)

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer = tf.keras.optimizers.Adam(learning_rate=0.001), metrics = ["accuracy"])

In [ ]:
print('running')
X = [X_train for _ in range(len(model.input))]

batch_size = 16
epochs = 100

history = model.fit(X, y_train,
                    batch_size=batch_size,
                    steps_per_epoch=len(X_train)//batch_size, 
                    epochs=epochs, 
                    callbacks = training_callbacks(),
                    validation_split=0.1)

# history = model.fit(X, y_train,
#                     batch_size=batch_size,
#                     steps_per_epoch=len(total_train) // batch_size,
#                     epochs=epochs, 
#                     validation_data=(X_1, y_test),
#                     validation_steps=len(total_val) // batch_size)